# Single-View Geometry (Python)

## Usage
This code snippet provides an overall code structure and some interactive plot interfaces for the *Single-View Geometry* section of Assignment 3. In [main function](#Main-function), we outline the required functionalities step by step. Some of the functions which involves interactive plots are already provided, but [the rest](#Your-implementation) are left for you to implement.

## Package installation
- You will need [GUI backend](https://matplotlib.org/faq/usage_faq.html#what-is-a-backend) to enable interactive plots in `matplotlib`.
- In this code, we use `tkinter` package. Installation instruction can be found [here](https://anaconda.org/anaconda/tk).

# Common imports

In [1]:
% matplotlib tk
import matplotlib.pyplot as plt
import numpy as np

from PIL import Image
import sympy as sp

# Provided functions

In [2]:
def get_input_lines(im, min_lines=3):
    """
    Allows user to input line segments; computes centers and directions.
    Inputs:
        im: np.ndarray of shape (height, width, 3)
        min_lines: minimum number of lines required
    Returns:
        n: number of lines from input
        lines: np.ndarray of shape (3, n)
            where each column denotes the parameters of the line equation
        centers: np.ndarray of shape (3, n)
            where each column denotes the homogeneous coordinates of the centers
    """
    n = 0
    lines = np.zeros((3, 0))
    centers = np.zeros((3, 0))

    plt.figure()
    plt.imshow(im)
    print('Set at least %d lines to compute vanishing point' % min_lines)
    while True:
        print('Click the two endpoints, use the right key to undo, and use the middle key to stop input')
        clicked = plt.ginput(2, timeout=0, show_clicks=True)
        if not clicked or len(clicked) < 2:
            if n < min_lines:
                print('Need at least %d lines, you have %d now' % (min_lines, n))
                continue
            else:
                # Stop getting lines if number of lines is enough
                break

        # Unpack user inputs and save as homogeneous coordinates
        pt1 = np.array([clicked[0][0], clicked[0][1], 1])
        pt2 = np.array([clicked[1][0], clicked[1][1], 1])
        # Get line equation using cross product
        # Line equation: line[0] * x + line[1] * y + line[2] = 0
        line = np.cross(pt1, pt2)
        lines = np.append(lines, line.reshape((3, 1)), axis=1)
        # Get center coordinate of the line segment
        center = (pt1 + pt2) / 2
        centers = np.append(centers, center.reshape((3, 1)), axis=1)

        # Plot line segment
        plt.plot([pt1[0], pt2[0]], [pt1[1], pt2[1]], color='b')

        n += 1

    return n, lines, centers

In [3]:
def plot_lines_and_vp(im, lines, vp):
    """
    Plots user-input lines and the calculated vanishing point.
    Inputs:
        im: np.ndarray of shape (height, width, 3)
        lines: np.ndarray of shape (3, n)
            where each column denotes the parameters of the line equation
        vp: np.ndarray of shape (3, )
    """
    bx1 = min(1, vp[0] / vp[2]) - 10
    bx2 = max(im.shape[1], vp[0] / vp[2]) + 10
    by1 = min(1, vp[1] / vp[2]) - 10
    by2 = max(im.shape[0], vp[1] / vp[2]) + 10

    plt.figure()
    plt.imshow(im)
    for i in range(lines.shape[1]):
        if lines[0, i] < lines[1, i]:
            pt1 = np.cross(np.array([1, 0, -bx1]), lines[:, i])
            pt2 = np.cross(np.array([1, 0, -bx2]), lines[:, i])
        else:
            pt1 = np.cross(np.array([0, 1, -by1]), lines[:, i])
            pt2 = np.cross(np.array([0, 1, -by2]), lines[:, i])
        pt1 = pt1 / pt1[2]
        pt2 = pt2 / pt2[2]
        plt.plot([pt1[0], pt2[0]], [pt1[1], pt2[1]], 'g')

    plt.plot(vp[0] / vp[2], vp[1] / vp[2], 'ro')
    plt.show()

In [4]:
def get_top_and_bottom_coordinates(im, obj):
    """
    For a specific object, prompts user to record the top coordinate and the bottom coordinate in the image.
    Inputs:
        im: np.ndarray of shape (height, width, 3)
        obj: string, object name
    Returns:
        coord: np.ndarray of shape (3, 2)
            where coord[:, 0] is the homogeneous coordinate of the top of the object and coord[:, 1] is the homogeneous
            coordinate of the bottom
    """
    plt.figure()
    plt.imshow(im)

    print('Click on the top coordinate of %s' % obj)
    clicked = plt.ginput(1, timeout=0, show_clicks=True)
    x1, y1 = clicked[0]
    # Uncomment this line to enable a vertical line to help align the two coordinates
    # plt.plot([x1, x1], [0, im.shape[0]], 'b')
    print('Click on the bottom coordinate of %s' % obj)
    clicked = plt.ginput(1, timeout=0, show_clicks=True)
    x2, y2 = clicked[0]

    plt.plot([x1, x2], [y1, y2], 'b')

    return np.array([[x1, x2], [y1, y2], [1, 1]])

# Your implementation

In [5]:
def get_vanishing_point(n, lines, centers):
    """
    Solves for the vanishing point using the user-input lines.
    """
    # <YOUR IMPLEMENTATION>
    # pairwise cross prod
    int1 = np.cross(lines[:, 0], lines[:, 1])
    int2 = np.cross(lines[:, 0], lines[:, 2])
    int3 = np.cross(lines[:, 1], lines[:, 2])

    int1 = int1 / int1[-1]
    int2 = int2 / int2[-1]
    int3 = int3 / int3[-1]
    
    vanishing_point = np.zeros((3, ))
    vanishing_point[0] = (int1[0] + int2[0] + int3[0]) / n
    vanishing_point[1] = (int1[1] + int2[1] + int3[1]) / n
    vanishing_point[2] = (int1[2] + int2[2] + int3[2]) / n
    
    return vanishing_point

In [6]:
def get_horizon_line(vpts):
    """
    Calculates the ground horizon line.
    """
    # <YOUR IMPLEMENTATION>
    x = vpts[:, 0]
    y = vpts[:, 1]
    a, b, c = np.cross(x, y)
    horizon_line = np.array([a, b, c])
    norm = np.sqrt(a ** 2 + b ** 2)
    horizon_line = horizon_line / norm
    return horizon_line

In [7]:
def plot_horizon_line(vpts, im):
    """
    Plots the horizon line.
    """
    # <YOUR IMPLEMENTATION>
    plt.figure()
    plt.imshow(im)
    v1, v2 = vpts[:, 0], vpts[:, 1]
    
    plt.plot([v1[0], v2[0]], [v1[1], v2[1]])
    plt.show()
    return


In [8]:
def get_camera_parameters(vpts):
    """
    Computes the camera parameters. Hint: The SymPy package is suitable for this.
    """
    # <YOUR IMPLEMENTATION>
    f = sp.Symbol('f')
    u = sp.Symbol('u')
    v = sp.Symbol('v')
    v1 = sp.Matrix(vpts[:, 0])
    v2 = sp.Matrix(vpts[:, 1])
    v3 = sp.Matrix(vpts[:, 2])
    
    inverse_K = sp.Matrix(((f, 0, u), (0, f, v), (0, 0, 1))).inv()
    
    e12 = v1.T * inverse_K.T * inverse_K * v2
    e13 = v1.T * inverse_K.T * inverse_K * v3
    e23 = v2.T * inverse_K.T * inverse_K * v3
    sol = sp.solve([e12, e13, e23], [f, u, v])
    f = sol[0][0]
    u = sol[0][1]
    v = sol[0][2]
    K = sp.Matrix(((f, 0, u), (0, f, v), (0, 0, 1)))
    return f, u, v, K

In [9]:
def get_rotation_matrix(vpts, K):
    Y_dir = vpts[:, 2]
    X_dir = vpts[:, 1]
    Z_dir = vpts[:, 0]
    
    inverse_K = np.array(K.inv()).astype(np.float)
    
    r1 = np.matmul(inverse_K, X_dir)
    r2 = np.matmul(inverse_K, Y_dir)
    r3 = np.matmul(inverse_K, Z_dir)
    
    r1 = r1 / np.linalg.norm(r1)
    r2 = r2 / np.linalg.norm(r2)
    r3 = r3 / np.linalg.norm(r3)
    
    R = np.zeros((3, 3))
    R[:, 0] = r1.T
    R[:, 1] = r2.T
    R[:, 2] = r3.T
    return R

In [10]:
def estimate_height(vpts, reference_height, reference_coord, object_coord):
    """
    Estimates height for a specific object using the recorded coordinates. You might need to plot additional images here for
    your report.
    """
    # <YOUR IMPLEMENTATION>
    vx = vpts[:, 0]
    vy = vpts[:, 1]
    vz = vpts[:, 2]
    r = object_coord[:, 0]
    t_0 = reference_coord[:, 0]
    b = object_coord[:, 1]
    b_0 = reference_coord[:, 1]
    v = np.cross(np.cross(b, b_0), np.cross(vx, vy))
    v = v / v[-1]
    t = np.cross(np.cross(v, t_0), np.cross(r, b))
    t = t / t[-1]
    height = reference_height \
    / ((np.linalg.norm(t - b) * np.linalg.norm(vz - r)) / (np.linalg.norm(r - b) * np.linalg.norm(vz - t)))
    return height

# Main function

In [11]:
im = np.asarray(Image.open('CSL.jpg'))

# Part 1
# Get vanishing points for each of the directions
num_vpts = 3
vpts = np.zeros((3, num_vpts))
for i in range(num_vpts):
    print('Getting vanishing point %d' % i)
    # Get at least three lines from user input
    n, lines, centers = get_input_lines(im)
    # <YOUR IMPLEMENTATION> Solve for vanishing point
    vpts[:, i] = get_vanishing_point(n, lines, centers)
    print('Coordinate of vanishing point:', vpts[:, i])
    # Plot the lines and the vanishing point
    plot_lines_and_vp(im, lines, vpts[:, i])

# <YOUR IMPLEMENTATION> Get the ground horizon line
horizon_line = get_horizon_line(vpts)
print
# <YOUR IMPLEMENTATION> Plot the ground horizon line
plot_horizon_line(vpts, im)

Getting vanishing point 0
Set at least 3 lines to compute vanishing point
Click the two endpoints, use the right key to undo, and use the middle key to stop input
Click the two endpoints, use the right key to undo, and use the middle key to stop input
Click the two endpoints, use the right key to undo, and use the middle key to stop input
Click the two endpoints, use the right key to undo, and use the middle key to stop input
Coordinate of vanishing point: [-289.03864625  216.61224449    1.        ]
Getting vanishing point 1
Set at least 3 lines to compute vanishing point
Click the two endpoints, use the right key to undo, and use the middle key to stop input
Click the two endpoints, use the right key to undo, and use the middle key to stop input
Click the two endpoints, use the right key to undo, and use the middle key to stop input
Click the two endpoints, use the right key to undo, and use the middle key to stop input
Coordinate of vanishing point: [1.34255615e+03 2.34926541e+02 1.0

In [12]:
# Part 2
# <YOUR IMPLEMENTATION> Solve for the camera parameters (f, u, v)
f, u, v, K = get_camera_parameters(vpts)
print('f: ', f)
print('u: ', u)
print('v: ', v)
print(K)

f:  -656.240348192502
u:  846.994041644131
v:  -138.108629845678
Matrix([[-656.240348192502, 0, 846.994041644131], [0, -656.240348192502, -138.108629845678], [0, 0, 1]])


In [13]:
# Part 3
# <YOUR IMPLEMENTATION> Solve for the rotation matrix
R = get_rotation_matrix(vpts, K)
print(R)

[[-0.54880217 -0.00979336  0.83589489]
 [-0.41311171  0.87247664 -0.26100426]
 [ 0.72674266  0.48855767  0.48286283]]


In [14]:
# Part 4
# Record image coordinates for each object and store in map
objects = ('person', 'CSL building', 'the spike statue', 'the lamp posts','person_l', 'person_m', 'person_r', 'fountain')
coords = dict()
for obj in objects:
    coords[obj] = get_top_and_bottom_coordinates(im, obj)

reference_height = 66.0 # inches
# <YOUR IMPLEMENTATION> Estimate heights
for obj in objects[1:]:
    print('Estimating height of %s' % obj)
    height = estimate_height(vpts, reference_height, coords['person'], coords[obj])
    print('Height of %s is: %f inches' % (obj, height))
    
reference_height = 72.0 # inches
# <YOUR IMPLEMENTATION> Estimate heights
for obj in objects[1:]:
    print('Estimating height of %s' % obj)
    height = estimate_height(vpts, reference_height, coords['person'], coords[obj])
    print('Height of %s is: %f inches' % (obj, height))

Click on the top coordinate of person
Click on the bottom coordinate of person
Click on the top coordinate of CSL building
Click on the bottom coordinate of CSL building
Click on the top coordinate of the spike statue
Click on the bottom coordinate of the spike statue
Click on the top coordinate of the lamp posts
Click on the bottom coordinate of the lamp posts
Click on the top coordinate of person_l
Click on the bottom coordinate of person_l
Click on the top coordinate of person_m
Click on the bottom coordinate of person_m
Click on the top coordinate of person_r
Click on the bottom coordinate of person_r
Click on the top coordinate of fountain
Click on the bottom coordinate of fountain
Estimating height of CSL building
Height of CSL building is: 1513.309880 inches
Estimating height of the spike statue
Height of the spike statue is: 431.123082 inches
Estimating height of the lamp posts
Height of the lamp posts is: 191.932377 inches
Estimating height of person_l
Height of person_l is: 6

In [15]:
print(horizon_line)

[-1.12240752e-02  9.99937008e-01 -2.19842791e+02]
